In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torchvision import models
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import numpy as np
from collections import defaultdict

def train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=10, num_folds=5):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(root=data_path, transform=transform)

    # Initialize K-Fold cross-validator
    kfold = KFold(n_splits=num_folds, shuffle=True)

    # Initialize containers for aggregated results
    all_preds = []
    all_labels = []

    # Start cross-validation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        # Sample elements randomly from a given list of indices for train and validation
        train_subsampler = Subset(dataset, train_ids)
        val_subsampler = Subset(dataset, val_ids)

        # Define data loaders for training and validation
        train_loader = DataLoader(dataset=train_subsampler, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_subsampler, batch_size=batch_size, shuffle=False)

        # Load a pre-trained AlexNet model
        model = models.alexnet(pretrained=True)

        # Modify the final layer to match the number of classes
        num_classes = len(dataset.classes)  # Automatically get the number of classes
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
        
        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        # Evaluation
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

    # Calculate and print the final aggregated classification report
    classes = dataset.classes  # Automatically get the class names from the dataset
    final_report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
    
    print('--------------------------------')
    print('Final Cross-Validation Results:')
    print(classification_report(all_labels, all_preds, target_names=classes))

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torchvision import models
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import numpy as np

def train_and_evaluate_AlexAdaGrad_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=10, num_folds=5):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(root=data_path, transform=transform)

    # Initialize K-Fold cross-validator
    kfold = KFold(n_splits=num_folds, shuffle=True)

    # Initialize containers for aggregated results
    all_preds = []
    all_labels = []

    # Start cross-validation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        # Sample elements randomly from a given list of indices for train and validation
        train_subsampler = Subset(dataset, train_ids)
        val_subsampler = Subset(dataset, val_ids)

        # Define data loaders for training and validation
        train_loader = DataLoader(dataset=train_subsampler, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_subsampler, batch_size=batch_size, shuffle=False)

        # Load a pre-trained AlexNet model
        model = models.alexnet(pretrained=True)

        # Modify the final layer to match the number of classes
        num_classes = len(dataset.classes)  # Automatically get the number of classes
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)  # Updated to AdaGrad optimizer

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        # Evaluation
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

    # Calculate and print the final aggregated classification report
    classes = dataset.classes  # Automatically get the class names from the dataset
    final_report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
    
    print('--------------------------------')
    print('Final Cross-Validation Results:')
    print(classification_report(all_labels, all_preds, target_names=classes))


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torchvision import models
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import numpy as np

def train_and_evaluate_AlexAdam_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=10, num_folds=5):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(root=data_path, transform=transform)

    # Initialize K-Fold cross-validator
    kfold = KFold(n_splits=num_folds, shuffle=True)

    # Initialize containers for aggregated results
    all_preds = []
    all_labels = []

    # Start cross-validation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        # Sample elements randomly from a given list of indices for train and validation
        train_subsampler = Subset(dataset, train_ids)
        val_subsampler = Subset(dataset, val_ids)

        # Define data loaders for training and validation
        train_loader = DataLoader(dataset=train_subsampler, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_subsampler, batch_size=batch_size, shuffle=False)

        # Load a pre-trained AlexNet model
        model = models.alexnet(pretrained=True)

        # Modify the final layer to match the number of classes
        num_classes = len(dataset.classes)  # Automatically get the number of classes
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Updated to Adam optimizer

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        # Evaluation
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

    # Calculate and print the final aggregated classification report
    classes = dataset.classes  # Automatically get the class names from the dataset
    final_report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
    
    print('--------------------------------')
    print('Final Cross-Validation Results:')
    print(classification_report(all_labels, all_preds, target_names=classes))

In [4]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Data"

train_and_evaluate_AlexAdam_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 2203.4472
Epoch [2/8], Loss: 2.3997
Epoch [3/8], Loss: 2.1890
Epoch [4/8], Loss: 2.1861
Epoch [5/8], Loss: 2.1774
Epoch [6/8], Loss: 2.1679
Epoch [7/8], Loss: 2.1598
Epoch [8/8], Loss: 2.1577
FOLD 2
--------------------------------
Epoch [1/8], Loss: 3801.7173
Epoch [2/8], Loss: 2.2000
Epoch [3/8], Loss: 2.1729
Epoch [4/8], Loss: 2.1692
Epoch [5/8], Loss: 2.1608
Epoch [6/8], Loss: 2.1653
Epoch [7/8], Loss: 2.1531
Epoch [8/8], Loss: 2.1544
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1134.0712
Epoch [2/8], Loss: 2.1539
Epoch [3/8], Loss: 2.1781
Epoch [4/8], Loss: 2.1615
Epoch [5/8], Loss: 2.1539
Epoch [6/8], Loss: 2.1429
Epoch [7/8], Loss: 2.1346
Epoch [8/8], Loss: 2.1414
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1185.0216
Epoch [2/8], Loss: 63.5113
Epoch [3/8], Loss: 29.8672
Epoch [4/8], Loss: 9.2550
Epoch [5/8], Loss: 2.1784
Epoch [6/8], Loss: 2.1630
Epoch [7/8], Loss: 2.1614
Epoch [8/8], Loss: 2.1552
FOLD 5
---------------------------

c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packag

In [5]:
train_and_evaluate_AlexAdam_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 2.0342
Epoch [2/8], Loss: 1.7665
Epoch [3/8], Loss: 1.7757
Epoch [4/8], Loss: 1.7794
Epoch [5/8], Loss: 1.8358
Epoch [6/8], Loss: 1.8007
Epoch [7/8], Loss: 1.6956
Epoch [8/8], Loss: 1.8756
FOLD 2
--------------------------------
Epoch [1/8], Loss: 2.1779
Epoch [2/8], Loss: 1.8908
Epoch [3/8], Loss: 1.7785
Epoch [4/8], Loss: 1.6687
Epoch [5/8], Loss: 1.5235
Epoch [6/8], Loss: 1.3907
Epoch [7/8], Loss: 1.5239
Epoch [8/8], Loss: 1.4798
FOLD 3
--------------------------------
Epoch [1/8], Loss: 2.2309
Epoch [2/8], Loss: 2.0112
Epoch [3/8], Loss: 1.7889
Epoch [4/8], Loss: 1.9826
Epoch [5/8], Loss: 1.8218
Epoch [6/8], Loss: 1.7630
Epoch [7/8], Loss: 1.8070
Epoch [8/8], Loss: 1.7328
FOLD 4
--------------------------------
Epoch [1/8], Loss: 2.2243
Epoch [2/8], Loss: 2.0106
Epoch [3/8], Loss: 2.4662
Epoch [4/8], Loss: 2.2199
Epoch [5/8], Loss: 2.2989
Epoch [6/8], Loss: 2.4729
Epoch [7/8], Loss: 2.3014
Epoch [8/8], Loss: 2.0657
FOLD 5
--------------------------------
Epoch [1

In [6]:
train_and_evaluate_AlexAdam_5fold(data_path, batch_size=16, learning_rate=0.0001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.4722
Epoch [2/8], Loss: 0.7874
Epoch [3/8], Loss: 0.5339
Epoch [4/8], Loss: 0.3868
Epoch [5/8], Loss: 0.3649
Epoch [6/8], Loss: 0.2423
Epoch [7/8], Loss: 0.1947
Epoch [8/8], Loss: 0.2443
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.6045
Epoch [2/8], Loss: 0.9626
Epoch [3/8], Loss: 0.6465
Epoch [4/8], Loss: 0.5067
Epoch [5/8], Loss: 0.4245
Epoch [6/8], Loss: 0.2955
Epoch [7/8], Loss: 0.2850
Epoch [8/8], Loss: 0.2458
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.5553
Epoch [2/8], Loss: 0.9366
Epoch [3/8], Loss: 0.6255
Epoch [4/8], Loss: 0.5232
Epoch [5/8], Loss: 0.3567
Epoch [6/8], Loss: 0.2810
Epoch [7/8], Loss: 0.2355
Epoch [8/8], Loss: 0.1961
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.4841
Epoch [2/8], Loss: 0.8831
Epoch [3/8], Loss: 0.5964
Epoch [4/8], Loss: 0.4541
Epoch [5/8], Loss: 0.3725
Epoch [6/8], Loss: 0.3094
Epoch [7/8], Loss: 0.2397
Epoch [8/8], Loss: 0.2482
FOLD 5
--------------------------------
Epoch [1

In [7]:
train_and_evaluate_AlexAdaGrad_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1257.4297
Epoch [2/8], Loss: 2.1640
Epoch [3/8], Loss: 2.1524
Epoch [4/8], Loss: 2.1492
Epoch [5/8], Loss: 2.1516
Epoch [6/8], Loss: 2.1464
Epoch [7/8], Loss: 2.1475
Epoch [8/8], Loss: 2.1444
FOLD 2
--------------------------------
Epoch [1/8], Loss: 2290.1423
Epoch [2/8], Loss: 2.1588
Epoch [3/8], Loss: 2.1595
Epoch [4/8], Loss: 2.1556
Epoch [5/8], Loss: 2.1580
Epoch [6/8], Loss: 2.1497
Epoch [7/8], Loss: 2.1487
Epoch [8/8], Loss: 2.1491
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1354.0086
Epoch [2/8], Loss: 2.2368
Epoch [3/8], Loss: 2.1650
Epoch [4/8], Loss: 2.1566
Epoch [5/8], Loss: 2.1576
Epoch [6/8], Loss: 2.1579
Epoch [7/8], Loss: 2.1559
Epoch [8/8], Loss: 2.1526
FOLD 4
--------------------------------
Epoch [1/8], Loss: 717.5272
Epoch [2/8], Loss: 2.8738
Epoch [3/8], Loss: 1.8479
Epoch [4/8], Loss: 1.6719
Epoch [5/8], Loss: 1.5525
Epoch [6/8], Loss: 1.3555
Epoch [7/8], Loss: 1.2507
Epoch [8/8], Loss: 1.1844
FOLD 5
------------------------------

c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packag

In [8]:
train_and_evaluate_AlexAdaGrad_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.9461
Epoch [2/8], Loss: 1.1605
Epoch [3/8], Loss: 0.8942
Epoch [4/8], Loss: 0.7241
Epoch [5/8], Loss: 0.5438
Epoch [6/8], Loss: 0.4459
Epoch [7/8], Loss: 0.3307
Epoch [8/8], Loss: 0.2898
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.9581
Epoch [2/8], Loss: 1.1602
Epoch [3/8], Loss: 0.8486
Epoch [4/8], Loss: 0.6268
Epoch [5/8], Loss: 0.4932
Epoch [6/8], Loss: 0.3501
Epoch [7/8], Loss: 0.2683
Epoch [8/8], Loss: 0.2416
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.9604
Epoch [2/8], Loss: 1.3653
Epoch [3/8], Loss: 1.0861
Epoch [4/8], Loss: 0.8926
Epoch [5/8], Loss: 0.7420
Epoch [6/8], Loss: 0.6549
Epoch [7/8], Loss: 0.5352
Epoch [8/8], Loss: 0.4525
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.8859
Epoch [2/8], Loss: 1.3804
Epoch [3/8], Loss: 1.0987
Epoch [4/8], Loss: 0.8975
Epoch [5/8], Loss: 0.7667
Epoch [6/8], Loss: 0.6368
Epoch [7/8], Loss: 0.4852
Epoch [8/8], Loss: 0.4125
FOLD 5
--------------------------------
Epoch [1

In [9]:
train_and_evaluate_AlexAdaGrad_5fold(data_path, batch_size=16, learning_rate=0.0001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.5908
Epoch [2/8], Loss: 1.1580
Epoch [3/8], Loss: 1.0113
Epoch [4/8], Loss: 0.9080
Epoch [5/8], Loss: 0.7757
Epoch [6/8], Loss: 0.7455
Epoch [7/8], Loss: 0.6500
Epoch [8/8], Loss: 0.5991
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.6371
Epoch [2/8], Loss: 1.1729
Epoch [3/8], Loss: 0.9861
Epoch [4/8], Loss: 0.8687
Epoch [5/8], Loss: 0.7956
Epoch [6/8], Loss: 0.7279
Epoch [7/8], Loss: 0.6721
Epoch [8/8], Loss: 0.6305
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.6230
Epoch [2/8], Loss: 1.1853
Epoch [3/8], Loss: 0.9874
Epoch [4/8], Loss: 0.8849
Epoch [5/8], Loss: 0.7849
Epoch [6/8], Loss: 0.6941
Epoch [7/8], Loss: 0.6414
Epoch [8/8], Loss: 0.5898
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.5997
Epoch [2/8], Loss: 1.1754
Epoch [3/8], Loss: 0.9688
Epoch [4/8], Loss: 0.8935
Epoch [5/8], Loss: 0.7712
Epoch [6/8], Loss: 0.7330
Epoch [7/8], Loss: 0.6710
Epoch [8/8], Loss: 0.5898
FOLD 5
--------------------------------
Epoch [1

In [7]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Data"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------
Epoch [1/8], Loss: 1.7124
Epoch [2/8], Loss: 1.2125
Epoch [3/8], Loss: 1.0112
Epoch [4/8], Loss: 0.8250
Epoch [5/8], Loss: 0.6824
Epoch [6/8], Loss: 0.5667
Epoch [7/8], Loss: 0.4887
Epoch [8/8], Loss: 0.5023
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.7375
Epoch [2/8], Loss: 1.1965
Epoch [3/8], Loss: 0.9521
Epoch [4/8], Loss: 0.8697
Epoch [5/8], Loss: 0.7331
Epoch [6/8], Loss: 0.6348
Epoch [7/8], Loss: 0.5546
Epoch [8/8], Loss: 0.5514
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.6507
Epoch [2/8], Loss: 1.2058
Epoch [3/8], Loss: 0.9311
Epoch [4/8], Loss: 0.8293
Epoch [5/8], Loss: 0.7050
Epoch [6/8], Loss: 0.5959
Epoch [7/8], Loss: 0.5764
Epoch [8/8], Loss: 0.4671
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.7272
Epoch [2/8], Loss: 1.2441
Epoch [3/8], Loss: 0.9923
Epoch [4/8], Loss: 0.7801
Epoch [5/8], Loss: 0.7212
Epoch [6/8], Loss: 0.6053
Epoch [7/8], Loss: 0.5657
Epoch [8/8], Loss: 0.5207
FOLD 5
-

In [8]:
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 2.0251
Epoch [2/8], Loss: 1.6183
Epoch [3/8], Loss: 1.4004
Epoch [4/8], Loss: 1.2819
Epoch [5/8], Loss: 1.1846
Epoch [6/8], Loss: 1.0766
Epoch [7/8], Loss: 1.0179
Epoch [8/8], Loss: 0.9531
FOLD 2
--------------------------------
Epoch [1/8], Loss: 2.0468
Epoch [2/8], Loss: 1.6040
Epoch [3/8], Loss: 1.4041
Epoch [4/8], Loss: 1.2215
Epoch [5/8], Loss: 1.1077
Epoch [6/8], Loss: 1.0007
Epoch [7/8], Loss: 0.9440
Epoch [8/8], Loss: 0.8920
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.9397
Epoch [2/8], Loss: 1.5704
Epoch [3/8], Loss: 1.3680
Epoch [4/8], Loss: 1.2277
Epoch [5/8], Loss: 1.1134
Epoch [6/8], Loss: 1.0511
Epoch [7/8], Loss: 0.9731
Epoch [8/8], Loss: 0.9325
FOLD 4
--------------------------------
Epoch [1/8], Loss: 2.0712
Epoch [2/8], Loss: 1.6103
Epoch [3/8], Loss: 1.3739
Epoch [4/8], Loss: 1.2298
Epoch [5/8], Loss: 1.1303
Epoch [6/8], Loss: 1.0515
Epoch [7/8], Loss: 0.9889
Epoch [8/8], Loss: 0.9552
FOLD 5
--------------------------------
Epoch [1

In [9]:
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.0001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 2.2307
Epoch [2/8], Loss: 2.0790
Epoch [3/8], Loss: 1.9997
Epoch [4/8], Loss: 1.9387
Epoch [5/8], Loss: 1.8814
Epoch [6/8], Loss: 1.8108
Epoch [7/8], Loss: 1.7627
Epoch [8/8], Loss: 1.7233
FOLD 2
--------------------------------
Epoch [1/8], Loss: 2.1633
Epoch [2/8], Loss: 2.0653
Epoch [3/8], Loss: 1.9909
Epoch [4/8], Loss: 1.9420
Epoch [5/8], Loss: 1.8944
Epoch [6/8], Loss: 1.8601
Epoch [7/8], Loss: 1.7989
Epoch [8/8], Loss: 1.7590
FOLD 3
--------------------------------
Epoch [1/8], Loss: 2.2879
Epoch [2/8], Loss: 2.1537
Epoch [3/8], Loss: 2.0771
Epoch [4/8], Loss: 1.9937
Epoch [5/8], Loss: 1.9401
Epoch [6/8], Loss: 1.8995
Epoch [7/8], Loss: 1.8407
Epoch [8/8], Loss: 1.7724
FOLD 4
--------------------------------
Epoch [1/8], Loss: 2.2306
Epoch [2/8], Loss: 2.1311
Epoch [3/8], Loss: 2.0604
Epoch [4/8], Loss: 1.9694
Epoch [5/8], Loss: 1.9131
Epoch [6/8], Loss: 1.8585
Epoch [7/8], Loss: 1.7887
Epoch [8/8], Loss: 1.7342
FOLD 5
--------------------------------
Epoch [1

c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\44778\anaconda3\Lib\site-packag

In [12]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Viridis\Mel (Raw)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------
Epoch [1/8], Loss: 1.6430
Epoch [2/8], Loss: 1.1923
Epoch [3/8], Loss: 0.9473
Epoch [4/8], Loss: 0.7154
Epoch [5/8], Loss: 0.6024
Epoch [6/8], Loss: 0.5243
Epoch [7/8], Loss: 0.5433
Epoch [8/8], Loss: 0.4138
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.7220
Epoch [2/8], Loss: 1.1475
Epoch [3/8], Loss: 0.9855
Epoch [4/8], Loss: 0.8218
Epoch [5/8], Loss: 0.7377
Epoch [6/8], Loss: 0.6443
Epoch [7/8], Loss: 0.5417
Epoch [8/8], Loss: 0.5322
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.5924
Epoch [2/8], Loss: 1.1493
Epoch [3/8], Loss: 0.9433
Epoch [4/8], Loss: 0.8124
Epoch [5/8], Loss: 0.6325
Epoch [6/8], Loss: 0.6089
Epoch [7/8], Loss: 0.5552
Epoch [8/8], Loss: 0.5485
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.6524
Epoch [2/8], Loss: 1.1690
Epoch [3/8], Loss: 0.9942
Epoch [4/8], Loss: 0.8359
Epoch [5/8], Loss: 0.7280
Epoch [6/8], Loss: 0.6628
Epoch [7/8], Loss: 0.5147
Epoch [8/8], Loss: 0.4895
FOLD 5
-

In [13]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Viridis\MFCC (Raw)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.7957
Epoch [2/8], Loss: 1.2195
Epoch [3/8], Loss: 0.9868
Epoch [4/8], Loss: 0.7958
Epoch [5/8], Loss: 0.7659
Epoch [6/8], Loss: 0.6650
Epoch [7/8], Loss: 0.5642
Epoch [8/8], Loss: 0.5425
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.8923
Epoch [2/8], Loss: 1.2876
Epoch [3/8], Loss: 1.1229
Epoch [4/8], Loss: 0.9812
Epoch [5/8], Loss: 0.8204
Epoch [6/8], Loss: 0.7340
Epoch [7/8], Loss: 0.6290
Epoch [8/8], Loss: 0.5538
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.8831
Epoch [2/8], Loss: 1.1972
Epoch [3/8], Loss: 1.0819
Epoch [4/8], Loss: 0.9110
Epoch [5/8], Loss: 0.8607
Epoch [6/8], Loss: 0.7405
Epoch [7/8], Loss: 0.6928
Epoch [8/8], Loss: 0.5752
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.8700
Epoch [2/8], Loss: 1.2215
Epoch [3/8], Loss: 0.9575
Epoch [4/8], Loss: 0.7802
Epoch [5/8], Loss: 0.7446
Epoch [6/8], Loss: 0.7885
Epoch [7/8], Loss: 0.5870
Epoch [8/8], Loss: 0.5173
FOLD 5
--------------------------------
Epoch [1

In [14]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Plasma\Mel (Raw)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.6620
Epoch [2/8], Loss: 1.1958
Epoch [3/8], Loss: 0.9925
Epoch [4/8], Loss: 0.9037
Epoch [5/8], Loss: 0.8086
Epoch [6/8], Loss: 0.7044
Epoch [7/8], Loss: 0.6313
Epoch [8/8], Loss: 0.5504
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.7086
Epoch [2/8], Loss: 1.2145
Epoch [3/8], Loss: 1.0419
Epoch [4/8], Loss: 0.9101
Epoch [5/8], Loss: 0.7880
Epoch [6/8], Loss: 0.6169
Epoch [7/8], Loss: 0.5910
Epoch [8/8], Loss: 0.5178
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.7494
Epoch [2/8], Loss: 1.3063
Epoch [3/8], Loss: 1.0181
Epoch [4/8], Loss: 0.8719
Epoch [5/8], Loss: 0.7411
Epoch [6/8], Loss: 0.6799
Epoch [7/8], Loss: 0.5830
Epoch [8/8], Loss: 0.5651
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.6936
Epoch [2/8], Loss: 1.2136
Epoch [3/8], Loss: 1.0513
Epoch [4/8], Loss: 0.9203
Epoch [5/8], Loss: 0.7951
Epoch [6/8], Loss: 0.6594
Epoch [7/8], Loss: 0.6286
Epoch [8/8], Loss: 0.5117
FOLD 5
--------------------------------
Epoch [1

In [15]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Plasma\MFCC (Raw)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.8536
Epoch [2/8], Loss: 1.2489
Epoch [3/8], Loss: 1.0837
Epoch [4/8], Loss: 1.0223
Epoch [5/8], Loss: 0.8653
Epoch [6/8], Loss: 0.8473
Epoch [7/8], Loss: 0.7670
Epoch [8/8], Loss: 0.6588
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.8511
Epoch [2/8], Loss: 1.4302
Epoch [3/8], Loss: 1.1336
Epoch [4/8], Loss: 1.0936
Epoch [5/8], Loss: 0.8620
Epoch [6/8], Loss: 0.7616
Epoch [7/8], Loss: 0.6912
Epoch [8/8], Loss: 0.6659
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.8146
Epoch [2/8], Loss: 1.3075
Epoch [3/8], Loss: 1.0645
Epoch [4/8], Loss: 0.8793
Epoch [5/8], Loss: 0.8435
Epoch [6/8], Loss: 0.7354
Epoch [7/8], Loss: 0.6481
Epoch [8/8], Loss: 0.5964
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.7583
Epoch [2/8], Loss: 1.3453
Epoch [3/8], Loss: 1.0972
Epoch [4/8], Loss: 0.9998
Epoch [5/8], Loss: 0.8417
Epoch [6/8], Loss: 0.7697
Epoch [7/8], Loss: 0.7660
Epoch [8/8], Loss: 0.6738
FOLD 5
--------------------------------
Epoch [1

In [16]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Cividis\Mel (Raw)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.6072
Epoch [2/8], Loss: 1.1462
Epoch [3/8], Loss: 0.8572
Epoch [4/8], Loss: 0.6784
Epoch [5/8], Loss: 0.5344
Epoch [6/8], Loss: 0.5112
Epoch [7/8], Loss: 0.4109
Epoch [8/8], Loss: 0.3949
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.6602
Epoch [2/8], Loss: 1.0872
Epoch [3/8], Loss: 0.8310
Epoch [4/8], Loss: 0.6963
Epoch [5/8], Loss: 0.5897
Epoch [6/8], Loss: 0.5342
Epoch [7/8], Loss: 0.4525
Epoch [8/8], Loss: 0.4021
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.6635
Epoch [2/8], Loss: 1.1177
Epoch [3/8], Loss: 0.9065
Epoch [4/8], Loss: 0.7510
Epoch [5/8], Loss: 0.6299
Epoch [6/8], Loss: 0.5660
Epoch [7/8], Loss: 0.4370
Epoch [8/8], Loss: 0.4315
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.6504
Epoch [2/8], Loss: 1.0497
Epoch [3/8], Loss: 0.8150
Epoch [4/8], Loss: 0.6657
Epoch [5/8], Loss: 0.6401
Epoch [6/8], Loss: 0.5054
Epoch [7/8], Loss: 0.4089
Epoch [8/8], Loss: 0.4512
FOLD 5
--------------------------------
Epoch [1

In [17]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Cividis\MFCC (Raw)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.7692
Epoch [2/8], Loss: 1.2008
Epoch [3/8], Loss: 1.0380
Epoch [4/8], Loss: 0.8884
Epoch [5/8], Loss: 0.7366
Epoch [6/8], Loss: 0.6816
Epoch [7/8], Loss: 0.5568
Epoch [8/8], Loss: 0.5604
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.8156
Epoch [2/8], Loss: 1.2760
Epoch [3/8], Loss: 1.0588
Epoch [4/8], Loss: 0.8772
Epoch [5/8], Loss: 0.8079
Epoch [6/8], Loss: 0.6956
Epoch [7/8], Loss: 0.6714
Epoch [8/8], Loss: 0.5527
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.7960
Epoch [2/8], Loss: 1.3300
Epoch [3/8], Loss: 1.1232
Epoch [4/8], Loss: 0.9535
Epoch [5/8], Loss: 0.8456
Epoch [6/8], Loss: 0.7395
Epoch [7/8], Loss: 0.7210
Epoch [8/8], Loss: 0.6589
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.8531
Epoch [2/8], Loss: 1.3570
Epoch [3/8], Loss: 1.1399
Epoch [4/8], Loss: 0.9535
Epoch [5/8], Loss: 0.7840
Epoch [6/8], Loss: 0.7950
Epoch [7/8], Loss: 0.6755
Epoch [8/8], Loss: 0.6285
FOLD 5
--------------------------------
Epoch [1